# Connect power network

- polygonise GridFinder targets raster
- attribute population and GDP to target areas
- connect centroids of targets to nearest line
- connect powerplants to nearest HV line

In [ ]:
import os
from glob import glob

import fiona
import geopandas as gpd
gpd._compat.USE_PYGEOS = False
import numpy as np
import numpy.ma
import pandas as pd
import rasterio
import rasterio.mask
import rasterio.features
import snkit
from pyproj import Geod
from rasterstats import zonal_stats, point_query
from shapely.geometry import shape

## Read power plants

In [ ]:
codes = ['KHM', 'IDN', 'LAO', 'MMR', 'PHL', 'THA', 'VNM']

In [ ]:
def get_powerplants(code):
    code = code.upper()
    
    powerplants = pd.read_csv('../incoming_data/globalpowerplantdatabasev120/global_power_plant_database.csv')
    powerplants = powerplants[powerplants.country == code].copy().reset_index(drop=True)
    powerplants['geometry'] = powerplants.apply(
        lambda row: shape({'type': 'Point', 'coordinates':[row.longitude, row.latitude]}),
        axis=1
    )
    powerplants = gpd.GeoDataFrame(
        powerplants[['gppd_idnr', 'name', 'capacity_mw', 'estimated_generation_gwh', 'primary_fuel', 'geometry']]
    )
    if code == 'MMR':
        # exclude two plants located in India/Bangladesh
        powerplants = powerplants[~powerplants.gppd_idnr.isin(['WRI1061379', 'WRI1061378'])]
    return powerplants

## Read targets, attribute population and GDP

In [ ]:
def get_target_areas(code):
    code = code.upper()
    
    with fiona.open(f"../incoming_data/Boundaries/Admin0/{code}_Admin0.gpkg", "r") as src:
        shapes = [feature["geometry"] for feature in src]

    geod = Geod(ellps="WGS84")

    with rasterio.open(f'../incoming_data/GridFinder/Targets/{code}_targets.asc') as src:

        data, transform = rasterio.mask.mask(src, shapes, crop=True)

        geoms = []
        centroids = []
        areas = []
        # Extract feature shapes and values from the array.
        for geom, val in rasterio.features.shapes(data, transform=transform):
            if val > 0:
                feature = shape(geom)
                geoms.append(feature)
                centroids.append(feature.centroid)
                area, perimeter = geod.geometry_area_perimeter(feature)
                areas.append(area / 1e6)

    return gpd.GeoDataFrame({'area_km2':areas, 'centroid':centroids, 'geometry':geoms})

In [ ]:
def get_population(code, targets):
    code = code.lower()
    
    populations = [
        d['nansum'] 
        for d in zonal_stats(
            targets.geometry,
            f'../incoming_data/Population/{code}_ppp_2020_1km_Aggregated_UNadj.tif',
            stats=[],
            add_stats={'nansum': np.nansum}, # count NaN as zero for summation
            all_touched=True # possible overestimate, but targets grid is narrower than pop
        )
    ]
    population_density = point_query(
        targets.centroid,
        f'../incoming_data/Population/{code}_ppp_2020_1km_Aggregated_UNadj.tif'
    )
    targets['population'] = populations
    targets['population_density_at_centroid'] = population_density
    
    def estimate_population_from_density(row):
        if row.population is numpy.ma.masked:
            return row.area_km2 * row.population_density_at_centroid
        else:
            return row.population

    targets['population'] = targets.apply(estimate_population_from_density, axis=1)
    
    return targets

In [ ]:
def get_gdp(code, targets):
    code = code.upper()
    
    # just pick centroid - GDP per capita doesn't vary at this fine granularity
    gdp_pc = point_query(
        targets.centroid,
        f'../incoming_data/GDP/{code}_GDP_per_capita_PPP_2015_v2.tif'
    )
    targets['gdp_pc'] = gdp_pc
    targets['gdp'] = targets.gdp_pc * targets.population
    return targets

## Read gridfinder lines

In [ ]:
def get_lines(code):
    code = code.upper()
    features = []
    with fiona.open(f'../incoming_data/GridFinder/{code}_GridFinder.gpkg') as src:
        for feature in src:
            # gridfinder GeoPackage stores an "fid" which GeoPandas ignores
            # and fiona reads as "id", not to feature['properties']
            # see https://github.com/geopandas/geopandas/issues/1035     
            geom = shape(feature['geometry'])
            features.append({
                'source_id': feature['id'],
                'source': feature['properties']['source'],
                'geometry': geom,
            })
            
    return gpd.GeoDataFrame(features)

## Set up network

In [ ]:
def patch_nearest_edge(point, edges):
    """Find nearest edge to a point
    """
    geom = point.buffer(1e-2) # include buffer to catch nearest line
    matches_idx = edges.sindex.nearest(geom.bounds)
    nearest_geom = min(
        [edges.iloc[match_idx] for match_idx in matches_idx],
        key=lambda match: point.distance(match.geometry)
    )
    return nearest_geom

snkit.network.nearest_edge = patch_nearest_edge

In [ ]:
for code in codes:
    print(code)
    # Power plants
    plants = get_powerplants(code) \
        .rename(columns={'gppd_idnr': 'source_id'})
    plants['type'] = 'source'
    plants['id'] = plants.reset_index()['index'].apply(lambda i: f"source_{code.lower()}_{i}")

    plants.to_file(
        f'../incoming_data/GridFinder/{code}_plants.gpkg',
        driver='GPKG'
    )

    plants = plants[['id', 'source_id', 'type', 'geometry']]
    
    # Targets
    targets = get_target_areas(code)
    targets = get_population(code, targets)
    targets = get_gdp(code, targets)
    targets = targets[~targets.gdp.isnull()].reset_index(drop=True)
    targets['type'] = 'target'
    targets['id'] = targets.reset_index()['index'].apply(lambda i: f"target_{code.lower()}_{i}")

    targets.drop(columns=['centroid']).to_file(
        f'../incoming_data/GridFinder/{code}_targets.gpkg',
        driver='GPKG'
    )

    targets = targets[['id', 'type', 'centroid']] \
        .rename(columns={'centroid': 'geometry'})
    
    # Combine to nodes
    nodes = plants.append(targets).reset_index(drop=True)
    
    # Edges
    edges = get_lines(code)
    edges['type'] = 'transmission'
    edges['id'] = targets.reset_index()['index'].apply(lambda i: f"plant_{code.lower()}_{i}")

    edges.to_file(
        f'../incoming_data/GridFinder/{code}_edges.gpkg',
        driver='GPKG'
    )

    edges = edges[['id', 'source_id', 'type', 'geometry']]
    
    # Process network
    network = snkit.network.Network(nodes, edges)
    network = snkit.network.split_multilinestrings(network)
    
    geod = Geod(ellps="WGS84")
    edge_limit = 20_000 # meters
    
    # Connect power plants
    network = snkit.network.link_nodes_to_nearest_edge(
        network, 
        lambda node, edge: node.type == 'source' and geod.geometry_length(edge.geometry) < edge_limit)
    network.nodes.loc[network.nodes.id.isnull(), 'type'] = 'conn_source'
    network.nodes['id'] = network.nodes.reset_index() \
        .apply(
            lambda row: f"conn_source_{code.lower()}_{row['index']}" if type(row.id) is float else row.id,
            axis=1
        )
    
    # Connect targets
    network = snkit.network.link_nodes_to_nearest_edge(
        network, 
        lambda node, edge: node.type == 'target' and geod.geometry_length(edge.geometry) < edge_limit)
    network.nodes.loc[network.nodes.id.isnull(), 'type'] = 'conn_target'
    network.nodes['id'] = network.nodes.reset_index() \
        .apply(
            lambda row: f"conn_target_{code.lower()}_{row['index']}" if type(row.id) is float else row.id,
            axis=1
        )
    
    # Add nodes at line endpoints
    network = snkit.network.add_endpoints(network)
    network.nodes.loc[network.nodes.id.isnull(), 'type'] = 'intermediate'
    network.nodes['id'] = network.nodes.reset_index() \
        .apply(
            lambda row: f"intermediate_{code.lower()}_{row['index']}" if type(row.id) is float else row.id,
            axis=1
        )

    # add from/to ids
    network = snkit.network.add_topology(network, id_col='id')
    
    # output
    out_fname = f'../incoming_data/GridFinder/{code}_network.gpkg'
    network.edges.to_file(out_fname, layer='edges', driver='GPKG')
    network.nodes.to_file(out_fname, layer='nodes', driver='GPKG')

## Postprocess to drop long edges

In [ ]:
for code in codes:
    print(code)
    fname = f'../incoming_data/GridFinder/{code}_network.gpkg'
    geod = Geod(ellps="WGS84")
    edge_limit = 20_000 # meters
    
    edges = gpd.read_file(fname, layer='edges')
    edges['length_m'] = edges.geometry.apply(geod.geometry_length)
    # keep only edges under the length limit, unless they're transmission edges from GridFinder
    edges = edges[(edges.length_m < edge_limit) | (edges['type'] == "transmission")]
    edges.to_file(fname, layer='edges', driver='GPKG')